<a href="https://colab.research.google.com/github/fatemafaria142/Exploring-Large-Language-Models-in-Science-Exam-Question-Answering/blob/main/Mistral_7B_Instruct_v0_2_fine_tuning_SciQ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Install Required Packages**

In [1]:
!pip install accelerate peft bitsandbytes transformers trl datasets torch -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 47.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ibis-framework 7.1.0 requires pyarrow<15,>=2, but you have pyarrow 15.0.0 which is in

#### **Dataset Link:** https://huggingface.co/datasets/sciq

In [2]:
from datasets import load_dataset

instruct_tune_dataset = load_dataset("sciq")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/11679 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

### **Dataset structure**
* The dataset contains train, test, validation seperately.

In [3]:
instruct_tune_dataset

DatasetDict({
    train: Dataset({
        features: ['question', 'distractor3', 'distractor1', 'distractor2', 'correct_answer', 'support'],
        num_rows: 11679
    })
    validation: Dataset({
        features: ['question', 'distractor3', 'distractor1', 'distractor2', 'correct_answer', 'support'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['question', 'distractor3', 'distractor1', 'distractor2', 'correct_answer', 'support'],
        num_rows: 1000
    })
})

In [4]:
# Display information for 5 data points from the 'train' split
num_samples_to_show = 5
for i in range(num_samples_to_show):
    data = instruct_tune_dataset['train'][i]
    print(f"Data Point {i + 1}:")
    print("Question:", data['question'])
    print("Response:", data['correct_answer'])
    print("\n-----------------------------\n")


Data Point 1:
Question: What type of organism is commonly used in preparation of foods such as cheese and yogurt?
Response: mesophilic organisms

-----------------------------

Data Point 2:
Question: What phenomenon makes global winds blow northeast to southwest or the reverse in the northern hemisphere and northwest to southeast or the reverse in the southern hemisphere?
Response: coriolis effect

-----------------------------

Data Point 3:
Question: Changes from a less-ordered state to a more-ordered state (such as a liquid to a solid) are always what?
Response: exothermic

-----------------------------

Data Point 4:
Question: What is the least dangerous radioactive decay?
Response: alpha decay

-----------------------------

Data Point 5:
Question: Kilauea in hawaii is the world’s most continuously active volcano. very active volcanoes characteristically eject red-hot rocks and lava rather than this?
Response: smoke and ash

-----------------------------



### **We will use just a small subset of the data for this training example**

In [5]:
instruct_tune_dataset["train"] = instruct_tune_dataset["train"].select(range(3500))
instruct_tune_dataset["test"] = instruct_tune_dataset["test"].select(range(500))

In [7]:
def create_prompt(sample):
    """
    Update the prompt template:
    Combine both the prompt and input into a single column.
    """
    bos_token = ""
    eos_token = ""

    # Use a predefined template for instructions
    instructions_template = "Put yourself in the shoes of a thoughtful science question-answer expert, ready to support and guide students. Imagine you're all set to help with understanding, prepared to offer support to students facing question-answer challenges"



    full_prompt = ""
    full_prompt += bos_token
    full_prompt += "### Instructions:"
    full_prompt += "\n" + instructions_template
    full_prompt += "\n\n### User:"
    full_prompt += "\n" + sample['question']
    full_prompt += "\n\n### Expert Response:"
    full_prompt += "\n" + sample['correct_answer']
    full_prompt += eos_token

    return full_prompt

In [8]:
create_prompt(instruct_tune_dataset["train"][0])

"### Instructions:\nPut yourself in the shoes of a thoughtful science question-answer expert, ready to support and guide students. Imagine you're all set to help with understanding, prepared to offer support to students facing question-answer challenges\n\n### User:\nWhat type of organism is commonly used in preparation of foods such as cheese and yogurt?\n\n### Expert Response:\nmesophilic organisms"

In [9]:
create_prompt(instruct_tune_dataset["train"][1])

"### Instructions:\nPut yourself in the shoes of a thoughtful science question-answer expert, ready to support and guide students. Imagine you're all set to help with understanding, prepared to offer support to students facing question-answer challenges\n\n### User:\nWhat phenomenon makes global winds blow northeast to southwest or the reverse in the northern hemisphere and northwest to southeast or the reverse in the southern hemisphere?\n\n### Expert Response:\ncoriolis effect"

In [10]:
create_prompt(instruct_tune_dataset["train"][2])

"### Instructions:\nPut yourself in the shoes of a thoughtful science question-answer expert, ready to support and guide students. Imagine you're all set to help with understanding, prepared to offer support to students facing question-answer challenges\n\n### User:\nChanges from a less-ordered state to a more-ordered state (such as a liquid to a solid) are always what?\n\n### Expert Response:\nexothermic"

In [11]:
create_prompt(instruct_tune_dataset["train"][3])

"### Instructions:\nPut yourself in the shoes of a thoughtful science question-answer expert, ready to support and guide students. Imagine you're all set to help with understanding, prepared to offer support to students facing question-answer challenges\n\n### User:\nWhat is the least dangerous radioactive decay?\n\n### Expert Response:\nalpha decay"

### **Initializing the Model**
* Load the model using a 4-bit configuration, employing double quantization, and set bfloat16 as the compute data type.

* Notably, we opt for the instruct-tuned model in this instance rather than the base model. It's worth mentioning that fine-tuning a base model necessitates a more substantial amount of data!

In [12]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

bnb_config = BitsAndBytesConfig(
        load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype="float16", bnb_4bit_use_double_quant=True
    )


* https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.1

In [13]:
mode_id = "mistralai/Mistral-7B-Instruct-v0.1"

In [14]:
model = AutoModelForCausalLM.from_pretrained(
        mode_id, quantization_config=bnb_config, device_map="auto", use_cache=False
    )

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [15]:
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

### **Let's example how well the model does at this task currently:**

In [17]:
def generate_response(prompt, model):
  encoded_input = tokenizer(prompt,  return_tensors="pt", add_special_tokens=True)
  model_inputs = encoded_input.to('cuda')

  generated_ids = model.generate(**model_inputs, max_new_tokens=256, do_sample=True, temperature = 0.5, pad_token_id=tokenizer.eos_token_id)

  decoded_output = tokenizer.batch_decode(generated_ids)

  return decoded_output[0].replace(prompt, "")

In [18]:
prompt = """### Instruction:
Put yourself in the shoes of a thoughtful science question-answer expert, ready to support and guide students. Imagine you're all set to help with understanding, prepared to offer support to students facing question-answer challenges.### User:
'''What phenomenon makes global winds blow northeast to southwest or the reverse in the northern hemisphere and northwest to southeast or the reverse in the southern hemisphere?'''

### Expert Response:"""

In [19]:
print(prompt)

### Instruction:
Put yourself in the shoes of a thoughtful science question-answer expert, ready to support and guide students. Imagine you're all set to help with understanding, prepared to offer support to students facing question-answer challenges.### User:
'''What phenomenon makes global winds blow northeast to southwest or the reverse in the northern hemisphere and northwest to southeast or the reverse in the southern hemisphere?'''

### Expert Response:


In [20]:
generate_response(prompt, model)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


"<s> \nThe phenomenon you are referring to is called the Coriolis effect, which is caused by the Earth's rotation. In the northern hemisphere, the Earth's rotation is counterclockwise, while in the southern hemisphere, it is clockwise. This causes the winds to blow in a circular pattern around the Earth, with the winds in the northern hemisphere blowing counterclockwise and the winds in the southern hemisphere blowing clockwise.\n\nThe Coriolis effect also causes the winds to spiral inward towards the equator, which is why the winds in the northern hemisphere blow from east to west and the winds in the southern hemisphere blow from west to east. This phenomenon is particularly noticeable in the trade winds, which blow from east to west across the equator and are responsible for the formation of the Hadley cells, which are large-scale atmospheric circulation patterns.\n\nIt's important to note that the Coriolis effect is not the only factor that influences wind patterns on Earth. Other 

### **Setting up the Training**
we will be using the `huggingface` and the `peft` library!

In [21]:
from peft import AutoPeftModelForCausalLM, LoraConfig, get_peft_model, prepare_model_for_kbit_training

peft_config = LoraConfig(r=8, lora_alpha=16, lora_dropout=0.05, bias="none", task_type="CAUSAL_LM")


* we need to prepare the model to be trained in 4bit so we will use the  **`prepare_model_for_kbit_training`** function from peft




In [22]:
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)

# **Training Hyperparameters**
The choice of hyperparameters is contingent upon the desired training duration. Pay special attention to the following key factors:

* `num_train_epochs/max_steps:` Dictates the number of iterations over the data. Exercise caution, as an excessive number may lead to overfitting!

* `learning_rate:` Governs the convergence speed of the model. Adjust this parameter judiciously for optimal results.

In [23]:
from transformers import TrainingArguments
output_model= "mistral_instruct_generation"
training_arguments = TrainingArguments(
        output_dir=output_model,
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        optim="paged_adamw_32bit",
        learning_rate=2e-4,
        lr_scheduler_type="cosine",
        save_strategy="epoch",
        logging_steps=10,
        num_train_epochs=1,
        max_steps=150,
        fp16=True,
)


### **Setting up the trainer**

`max_seq_length`: Context window size


In [24]:
from trl import SFTTrainer

max_seq_length = 256

trainer = SFTTrainer(
  model=model,
  peft_config=peft_config,
  max_seq_length=max_seq_length,
  tokenizer=tokenizer,
  packing=True,
  formatting_func=create_prompt, # this will aplly the create_prompt mapping to all training and test dataset
  args=training_arguments,
  train_dataset=instruct_tune_dataset["train"],
  eval_dataset=instruct_tune_dataset["test"]
)

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:310: UserWarning: You passed `packing=True` to the SFTTrainer, and you are training your model with `max_steps` strategy. The dataset will be iterated until the `max_steps` are reached.
  warnings.warn(


### **Training starts here**

In [25]:
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
10,1.450900
20,0.862200
30,0.629500
40,0.623200
50,0.606500
60,0.589200
70,0.590000
80,0.573500
90,0.560400
100,0.566900


TrainOutput(global_step=150, training_loss=0.6558158969879151, metrics={'train_runtime': 852.6142, 'train_samples_per_second': 0.704, 'train_steps_per_second': 0.176, 'total_flos': 6556325039308800.0, 'train_loss': 0.6558158969879151, 'epoch': 0.52})

### **Save the model**

In [26]:
trainer.save_model("mistral_instruct_generation")

In [27]:
merged_model = model.merge_and_unload()

/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/bnb.py:249: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


### **Example No:1**

In [28]:
# Example usage
prompt = '''### Instructions:\n Put yourself in the shoes of a thoughtful science question-answer expert, ready to support and guide students. Imagine you're all set to help with understanding, prepared to offer support to students facing question-answer challenges.'''
prompt += '''\n### User:\nWhat chemical signals in plants control different processes?'''
prompt += "\n\n### Expert Response:"
response = generate_response(prompt, merged_model)

# Print the response with formatted output
print(response)

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


<s> 
hormones</s>


### **Example No:2**

In [29]:
# Example usage
prompt = '''### Instructions:\n Put yourself in the shoes of a thoughtful science question-answer expert, ready to support and guide students. Imagine you're all set to help with understanding, prepared to offer support to students facing question-answer challenges.'''
prompt += '''\n### User:\nPenis, testes, and epididymis are organs in what system?'''
prompt += "\n\n### Expert Response:"
response = generate_response(prompt, merged_model)

# Print the response with formatted output
print(response)

<s> 
reproductive system</s>


### **Example No:3**

In [30]:
# Example usage
prompt = '''### Instructions:\n Put yourself in the shoes of a thoughtful science question-answer expert, ready to support and guide students. Imagine you're all set to help with understanding, prepared to offer support to students facing question-answer challenges.'''
prompt += '''\n### User:\nOnly about one percent of plants have lost what ability, turning them into consumers and even predators, instead of producers?'''
prompt += "\n\n### Expert Response:"
response = generate_response(prompt, merged_model)

# Print the response with formatted output
print(response)

<s> 
photosynthesis</s>


### **Example No:4**

In [33]:
# Example usage
prompt = '''### Instructions:\n Put yourself in the shoes of a thoughtful science question-answer expert, ready to support and guide students. Imagine you're all set to help with understanding, prepared to offer support to students facing question-answer challenges.'''
prompt += '''\n### User:\nBy the year 2050, 25 percent of the population of the united states will be 60 years of age or older. the cdc estimates that 80 percent of those 60 years and older have one or more chronic disease associated with deficiencies of this?'''
prompt += "\n\n### Expert Response:"
response = generate_response(prompt, merged_model)

# Print the response with formatted output
print(response)

<s> 
vitamins</s>


### **Example No:5**

In [32]:
# Example usage
prompt = '''### Instructions:\n Put yourself in the shoes of a thoughtful science question-answer expert, ready to support and guide students. Imagine you're all set to help with understanding, prepared to offer support to students facing question-answer challenges.'''
prompt += '''\n### User:\nThe lens focuses light on the retina , which covers the back of the inside of the eye. the retina has light-sensing photoreceptor cells called?'''
prompt += "\n\n### Expert Response:"
response = generate_response(prompt, merged_model)

# Print the response with formatted output
print(response)

<s> 
rods and cones</s>
